# Introduction to the Monte Carlo method

----

Start by defining the [Gibbs (or Boltzmann) distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution):
$$P(\alpha) = e^{-E(\alpha)/kT}$$
this expression, defines the probability of observing a particular configuration of spins, $\alpha$. 
As you can see, the probability of $\alpha$ decays exponentially with increasing energy of $\alpha$, $E(\alpha)$,
where $k$ is the Boltzmann constant, $k = 1.38064852 \times 10^{-23} J/K$
and $T$ is the temperature in Kelvin. 

## What defines the energy of a configuration of spins? 
Given a configuration of spins (e.g., $\uparrow\downarrow\downarrow\uparrow\downarrow$) we can define the energy using what is referred to as an Ising Hamiltonian:
$$ \hat{H}' = \frac{\hat{H}}{k} = -\frac{J}{k}\sum_{<ij>} s_is_j,$$
where, $s_i=1$ if the $i^{th}$ spin is `up` and $s_i=-1$ if it is `down`, and the brackets $<ij>$ indicate a sum over spins that are connected,
and $J$ is a constant that determines the energy scale. 
The energy here has been divided by the Boltzmann constant to yield units of temperature. 
Let's consider the following case, which has the sites connected in a single 1D line:
$$\alpha = \uparrow-\downarrow-\downarrow-\uparrow-\downarrow.$$ 
What is the energy of such a configuration?
$$ E(\alpha)' = J/k(-1 + 1 - 1 - 1) = \frac{E(\alpha)}{k} = -2J/k$$

## P1: Write a class that defines a spin configuration

In [3]:
class SpinConfig :
    def __init__(self, spins) :
        assert(all(s == 1 or s == -1 for s in spins))
        self.__spins = list(spins)
        self.__len = len(spins)
    def __len__(self) :
        return self.__len
    def __str__(self) :
        return str(self.__spins)
    def __getitem__(self, index) :
        return self.__spins[index]
    def __setitem__(self, index, value) :
        assert(value == 1 or value == -1)
        self.__spins[index] = value
        return value
    def __iter__(self) :
        return iter(self.__spins)
    

## P2: Write a class that defines the 1D hamiltonian, containing a function that computes the energy of a configuration

In [1]:
boltzmann_k = 1.38064852e-23
class Hamiltonian :
    def __init__(self, coupling, magnet, spinconf) :
        self.__spins = spinconf
        self.__coupling = coupling
        self.__magnet = magnet
    def getspins(self) :
        return self.__spins
    def getcoupling(self) :
        return self.__coupling
    def getmagnet(self) :
        return self.__magnet
    def findenergy(self) :
        return -self.__coupling * sum(self.__spins[i - 1] * self.__spins[i] 
                                                   for i in range(len(self.__spins))) + \
            self.__magnet * sum(self.__spins)
    def findtemp(self) :
        return self.findenergy() / boltzmann_k
    

## Properties
For any fixed state, $\alpha$, the `magnetization` ($M$) is proportional to the _excess_ number of spins pointing up or down while the energy is given by the
Hamiltonian:
$$M(\alpha) = N_{\text{up}}(\alpha) - N_{\text{down}}(\alpha).$$
As a dynamical, fluctuating system, each time you measure the magnetization, the system might be in a different state ($\alpha$) and so you'll get a different number!
However, we already know what the probability of measuring any particular $\alpha$ is, so in order to compute the average magnetization, $\left<M\right>$, we just need to multiply the magnetization of each possible configuration times the probability of it being measured, and then add them all up!
$$ \left<M\right> = \sum_\alpha M(\alpha)P(\alpha).$$
In fact, any average value can be obtained by adding up the value of an individual configuration multiplied by it's probability:
$$ \left<E\right> = \sum_\alpha E(\alpha)P(\alpha).$$

This means that to obtain any average value (also known as an `expectation value`) computationally, we must compute the both the value and probability of all possible configurations. This becomes extremely expensive as the number of spins ($N$) increases. 

## P3: Write a function that computes the magnetization of a spin configuration

In [4]:
def magnetization(spinconf) :
    return len(list(filter(s == 1 for s in spinconf))) - len(list(filter(s == -1 for s in spinconf)))

## Q2: How many configurations are possible for:

(a) N=10?

In [5]:
def numconf(sites) :
    if type(sites) is int :
        return 2 ** sites
    else :
        return 2 ** len(sites)
numconf(10)

1024

(b) N=100?

In [6]:
numconf(100)

1267650600228229401496703205376

(c) N=1000?

## Q3: What is the energy for (++-+---+--+)?

In [6]:
config = SpinConfig([1, 1, -1, 1, -1, -1, -1, 1, -1, -1, 1])
ham = Hamiltonian(2, 1.1, config)
ham.findenergy()

0.8999999999999999